# Comparación de modelos de detección de odio

Este notebook:

- Lee todos los ficheros `.json` de `data/results/`.
- Extrae las métricas principales de cada modelo.
- Construye una tabla comparativa y ordena los modelos por `f1` (y opcionalmente por `roc_auc`).
- Opcionalmente, guarda un resumen en `data/results/model_comparison.csv`.

Todos los notebooks de modelado deben generar un `.json` con la siguiente estructura:

```json
{
  "model_name": "logistic_regression_toxic_v1",
  "task": "binary_classification",
  "target_label": "IsToxic",
  "data": {
    "n_samples": 997,
    "n_features_text": 10000,
    "n_features_numeric": 5,
    "train_size": 0.8,
    "test_size": 0.2,
    "random_state": 42
  },
  "metrics": {
    "accuracy": 0.87,
    "precision": 0.85,
    "recall": 0.83,
    "f1": 0.84,
    "roc_auc": 0.91
  },
  "confusion_matrix": {
    "tn": 210,
    "fp": 30,
    "fn": 35,
    "tp": 222
  },
  "timestamp": "2025-11-27T21:45:00",
  "notes": "TF-IDF (1,2), max_features=10000; text_classic + 5 numeric features"
}


### 🧩 Celda 2 – Código (imports y localización de carpeta)

In [1]:
# =============================================================================
# 1. IMPORTS Y LOCALIZACIÓN DE LA CARPETA DE RESULTADOS
# =============================================================================

import json  # To read JSON results
from pathlib import Path  # To handle filesystem paths

import pandas as pd  # To build comparison tables
import numpy as np  # Optional, for numeric operations
import matplotlib.pyplot as plt  # Optional, for simple plots

# Detect project root
notebook_dir = Path.cwd()
if "notebooks" in str(notebook_dir):
    project_root = notebook_dir.parent.parent
else:
    project_root = notebook_dir

results_dir = project_root / "data" / "results"

print(f"📂 Raíz del proyecto  : {project_root}")
print(f"📁 Carpeta de resultados: {results_dir}")


📂 Raíz del proyecto  : c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4
📁 Carpeta de resultados: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\data\results


In [2]:
# =============================================================================
# 2. CARGA DE RESULTADOS DESDE data/results/*.json
# =============================================================================

if not results_dir.exists():
    raise FileNotFoundError(f"❌ La carpeta {results_dir} no existe. Asegúrate de haber guardado los JSON primero.")

json_files = sorted(results_dir.glob("*.json"))

if not json_files:
    raise FileNotFoundError(f"❌ No se han encontrado ficheros .json en {results_dir}")

print("✅ Ficheros de resultados encontrados:")
for f in json_files:
    print(f"   - {f.name}")

rows = []

for fpath in json_files:
    with open(fpath, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    row = {
        "model_name": data.get("model_name"),
        "task": data.get("task"),
        "target_label": data.get("target_label"),
        "n_samples": data.get("data", {}).get("n_samples"),
        "n_features_text": data.get("data", {}).get("n_features_text"),
        "n_features_numeric": data.get("data", {}).get("n_features_numeric"),
        "train_size": data.get("data", {}).get("train_size"),
        "test_size": data.get("data", {}).get("test_size"),
        "accuracy": data.get("metrics", {}).get("accuracy"),
        "precision": data.get("metrics", {}).get("precision"),
        "recall": data.get("metrics", {}).get("recall"),
        "f1": data.get("metrics", {}).get("f1"),
        "roc_auc": data.get("metrics", {}).get("roc_auc"),
        "tn": data.get("confusion_matrix", {}).get("tn"),
        "fp": data.get("confusion_matrix", {}).get("fp"),
        "fn": data.get("confusion_matrix", {}).get("fn"),
        "tp": data.get("confusion_matrix", {}).get("tp"),
        "timestamp": data.get("timestamp"),
        "notes": data.get("notes", ""),
    }
    rows.append(row)

df_results = pd.DataFrame(rows)

print("\n📊 Tabla de resultados (sin ordenar):")
display(df_results)


✅ Ficheros de resultados encontrados:
   - naive_bayes_toxic_v1.json
   - svm_toxic_v1.json

📊 Tabla de resultados (sin ordenar):


,model_name,task,target_label,n_samples,n_features_text,n_features_numeric,train_size,test_size,accuracy,precision,recall,f1,roc_auc,tn,fp,fn,tp,timestamp,notes
0,naive_bayes_toxic_v1,binary_classification,IsToxic,997,2065,5,0.799398,0.200602,0.760,0.833333,0.597826,0.696203,0.801429,97,11,37,55,2025-11-28T02:11:06,"Naive Bayes + TF-IDF (1,2) + 5 numeric feature..."
1,svm_toxic_v1,binary_classification,IsToxic,997,1136,5,0.799398,0.200602,0.705,0.694118,0.641304,0.666667,0.760668,82,26,33,59,2025-12-01T09:00:24.702598,"LinearSVC + TF-IDF (1,2) + 5 numeric features ..."


In [3]:
# =============================================================================
# 3. ORDENAR MODELOS POR F1 Y ROC-AUC
# =============================================================================

# Sort by F1 (descending), then by ROC-AUC
df_sorted = df_results.sort_values(by=["f1", "roc_auc"], ascending=False)

print("🏆 Modelos ordenados por F1 y ROC-AUC:")
display(df_sorted[
    [
        "model_name",
        "target_label",
        "accuracy",
        "precision",
        "recall",
        "f1",
        "roc_auc",
        "n_features_text",
        "n_features_numeric",
        "timestamp",
    ]
])

# Best model overall
best_model = df_sorted.iloc[0]
print("\n🥇 Mejor modelo global (por F1):")
print(f"   Modelo       : {best_model['model_name']}")
print(f"   Target label : {best_model['target_label']}")
print(f"   F1           : {best_model['f1']:.3f}")
print(f"   ROC-AUC      : {best_model['roc_auc']:.3f}")


🏆 Modelos ordenados por F1 y ROC-AUC:


,model_name,target_label,accuracy,precision,recall,f1,roc_auc,n_features_text,n_features_numeric,timestamp
0,naive_bayes_toxic_v1,IsToxic,0.760,0.833333,0.597826,0.696203,0.801429,2065,5,2025-11-28T02:11:06
1,svm_toxic_v1,IsToxic,0.705,0.694118,0.641304,0.666667,0.760668,1136,5,2025-12-01T09:00:24.702598



🥇 Mejor modelo global (por F1):
   Modelo       : naive_bayes_toxic_v1
   Target label : IsToxic
   F1           : 0.696
   ROC-AUC      : 0.801


In [4]:
# =============================================================================
# 4. GUARDAR RESUMEN EN CSV (OPCIONAL)
# =============================================================================

summary_path = results_dir / "model_comparison.csv"
df_sorted.to_csv(summary_path, index=False)

print(f"💾 Resumen de comparación guardado en: {summary_path}")


💾 Resumen de comparación guardado en: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\data\results\model_comparison.csv
